In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
from shutil import copy
from zipfile import ZipFile
import matplotlib.pyplot as plt
import cv2
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from matplotlib import pyplot as plt
from matplotlib import image
from sklearn.model_selection import KFold

In [ ]:
data_path = "/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/"

In [ ]:
train_set = data_path + 'train/'
val_set = data_path + 'valid/'
test_set = data_path + 'test/'
epochs = 30

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1,
    rescale=1. / 255
)
val_datagen= ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1,
    rescale=1. / 255)
test_datagen= ImageDataGenerator(
    rescale=1. / 255
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_set,
    target_size = (224,224),
    batch_size = 8,
    class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory(
    test_set,
    target_size = (224,224),
    batch_size = 8,
    shuffle=True,
    class_mode = 'categorical')
validation_generator = test_datagen.flow_from_directory(
    val_set,
    target_size = (224,224),
    batch_size = 8,
    shuffle=True,
    class_mode = 'categorical')

Found 613 images belonging to 4 classes.
Found 315 images belonging to 4 classes.
Found 72 images belonging to 4 classes.


In [ ]:
from keras.applications import MobileNet, DenseNet169, ResNet50, EfficientNetB2, EfficientNetB4, Xception, InceptionV3, VGG19, DenseNet201, ResNet152
from keras.layers import GlobalAveragePooling2D

In [ ]:
# base_model = MobileNet(weights='imagenet', input_shape=(224,224,3), include_top=False)
# base_model = DenseNet169(weights='imagenet', input_shape=(224,224,3), include_top=False)
# base_model = EfficientNetB2(weights='imagenet', input_shape=(224,224,3), include_top=False)
# base_model = EfficientNetB4(weights='imagenet', input_shape=(224,224,3), include_top=False)
# base_model = ResNet50(weights='imagenet', input_shape=(224,224,3), include_top=False)
# base_model = Xception(weights='imagenet', input_shape=(224,224,3), include_top=False)
# base_model = InceptionV3(weights='imagenet', input_shape=(224,224,3), include_top=False)
# base_model = VGG19(weights='imagenet', input_shape=(224,224,3), include_top=False)
# base_model = DenseNet201(weights='imagenet', input_shape=(224,224,3), include_top=False)
base_model = ResNet152(weights='imagenet', input_shape=(224,224,3), include_top=False)

for layer in base_model.layers:
    layer.trainable = False

last_layer = base_model.output
last_layer = BatchNormalization()(last_layer)
last_layer = GlobalAveragePooling2D()(last_layer)
last_layer = Dense(256,activation='relu')(last_layer)
last_layer = Dense(64,activation='relu')(last_layer)
last_layer = Dense(16,activation='relu')(last_layer)
last_layer = Dropout(0.4)(last_layer)
last_layer = Dense(4,activation='sigmoid')(last_layer)
model = Model(base_model.input, last_layer)

234698864/234698864 [==============================] - 11s 0us/step


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy','Precision','Recall'])


model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_5[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [ ]:
# filepath =  '/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'MobileNet/' + 'saved-model-{epoch:02d}-acc_{val_accuracy:.4f}-pre_{val_precision:.4f}-rec_{val_recall:.4f}.hdf5'
# filepath =  '/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'DenseNet169/' + 'saved-model-{epoch:02d}-acc_{val_accuracy:.4f}-pre_{val_precision:.4f}-rec_{val_recall:.4f}.hdf5'
# filepath =  '/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'EfficientNetB2/' + 'saved-model-{epoch:02d}-acc_{val_accuracy:.4f}-pre_{val_precision:.4f}-rec_{val_recall:.4f}.hdf5'
# filepath =  '/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'EfficientNetB4/' + 'saved-model-{epoch:02d}-acc_{val_accuracy:.4f}-pre_{val_precision:.4f}-rec_{val_recall:.4f}.hdf5'
# filepath =  '/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'ResNet50/' + 'saved-model-{epoch:02d}-acc_{val_accuracy:.4f}-pre_{val_precision:.4f}-rec_{val_recall:.4f}.hdf5'
# filepath =  '/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'Xception/' + 'saved-model-{epoch:02d}-acc_{val_accuracy:.4f}-pre_{val_precision:.4f}-rec_{val_recall:.4f}.hdf5'
# filepath =  '/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'InceptionV3/' + 'saved-model-{epoch:02d}-acc_{val_accuracy:.4f}-pre_{val_precision:.4f}-rec_{val_recall:.4f}.hdf5'
# filepath =  '/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'VGG19/' + 'saved-model-{epoch:02d}-acc_{val_accuracy:.4f}-pre_{val_precision:.4f}-rec_{val_recall:.4f}.hdf5'
# filepath =  '/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'DenseNet201/' + 'saved-model-{epoch:02d}-acc_{val_accuracy:.4f}-pre_{val_precision:.4f}-rec_{val_recall:.4f}.hdf5'
filepath =  '/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'ResNet152/' + 'saved-model-{epoch:02d}-acc_{val_accuracy:.4f}-pre_{val_precision:.4f}-rec_{val_recall:.4f}.hdf5'

checkpoint = ModelCheckpoint(filepath,
                             monitor='val_accuracy',
                             save_best_only=True,
                             mode='max')

earlystop = EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          patience=10,
                          restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                            patience=5,
                                            verbose=1,
                                            factor=0.2,
                                            min_lr=0.0001)

my_callbacks = [checkpoint,earlystop,learning_rate_reduction]

In [ ]:
history = model.fit_generator(
    train_generator,
    epochs = epochs,
    validation_data = validation_generator,
    callbacks = my_callbacks
)

Epoch 1/30


<ipython-input-38-b0e91f9a723d>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


77/77 [==============================] - 26s 238ms/step - loss: 1.1568 - accuracy: 0.4666 - precision: 0.3944 - recall: 0.7064 - val_loss: 1.0911 - val_accuracy: 0.4861 - val_precision: 0.4161 - val_recall: 0.7917 - lr: 0.0010
Epoch 2/30
77/77 [==============================] - 15s 189ms/step - loss: 1.0855 - accuracy: 0.5481 - precision: 0.3918 - recall: 0.7945 - val_loss: 1.0591 - val_accuracy: 0.4583 - val_precision: 0.3651 - val_recall: 0.9583 - lr: 0.0010
Epoch 3/30
77/77 [==============================] - 13s 168ms/step - loss: 1.0297 - accuracy: 0.5563 - precision: 0.4014 - recall: 0.8140 - val_loss: 1.0144 - val_accuracy: 0.4583 - val_precision: 0.3784 - val_recall: 0.9722 - lr: 0.0010
Epoch 4/30
77/77 [==============================] - 13s 164ms/step - loss: 1.0067 - accuracy: 0.5661 - precision: 0.3913 - recall: 0.8336 - val_loss: 1.0136 - val_accuracy: 0.4861 - val_precision: 0.3730 - val_recall: 0.9583 - lr: 0.0010
Epoch 5/30
77/77 [==============================] - 15s 191

KeyboardInterrupt: ignored

In [ ]:
t = model.predict(test_generator)

40/40 [==============================] - 206s 5s/step


In [ ]:
# true_labels = test_generator.classes
# predictions = model.predict_generator(test_generator)

In [ ]:
# y_true = true_labels
# y_pred = np.array([np.argmax(x) for x in predictions])

In [ ]:
# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(y_true, y_pred)

In [ ]:
# print(cm)

In [ ]:
# import seaborn as sn
# import pandas as pd
# import matplotlib.pyplot as plt

# df_cm = pd.DataFrame(cm, index = [i for i in "ABCD"],
#                   columns = [i for i in "ABCD"])
# plt.figure(figsize = (10,7))
# sn.heatmap(df_cm, annot=True)

In [ ]:
a = model.evaluate(test_generator)

In [ ]:
# filepath =  '/content/' + 'MobileNet/'

import pickle

# some_obj = {'loss':a[0], 'accuracy':a[1], 'precision':a[2], 'recall':a[3], 'cm':cm}
some_obj = {'loss':a[0], 'accuracy':a[1], 'precision':a[2], 'recall':a[3]}
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'MobileNet/' +'metrics.pickle', 'wb') as f:
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'DenseNet169/' +'metrics.pickle', 'wb') as f:
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'EfficientNetB2/' +'metrics.pickle', 'wb') as f:
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'EfficientNetB4/' +'metrics.pickle', 'wb') as f:
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'ResNet50/' +'metrics.pickle', 'wb') as f:
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'Xception/' +'metrics.pickle', 'wb') as f:
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'InceptionV3/' +'metrics.pickle', 'wb') as f:
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'VGG19/' +'metrics.pickle', 'wb') as f:
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'DenseNet201/' +'metrics.pickle', 'wb') as f:
with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'ResNet152/' +'metrics.pickle', 'wb') as f:
    pickle.dump(some_obj, f)

In [ ]:
some_obj

{'loss': 1.006168246269226,
 'accuracy': 0.5523809790611267,
 'precision': 0.4612612724304199,
 'recall': 0.8126984238624573}